In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#PDFs in BDT and sindec?
import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
from tqdm.notebook import tqdm_notebook as tqdm
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
selection_version = 'version-001-p00'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    data_prefix = '/data/user/ssclafani/data/cscd/final'
    ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = '/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/pointing_resolution'
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository()
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [ ]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

In [ ]:
a = ana.anas[0]
a.sig

In [ ]:
a.bg_data

## Helpers

In [ ]:
from cycler import cycle
from copy import deepcopy

soft_colors = cy.plotting.soft_colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


def get_bias_allt(tr, ntrials=200, n_sigs=np.r_[:101:10], quiet=False):
    trials = [
        (None if quiet else print(f'\r{n_sig:4d} ...', end='', flush=True))
        or
        tr.get_many_fits(ntrials, n_sig=n_sig, logging=False, seed=n_sig)
        for n_sig in n_sigs]
    if not quiet:
        print()
    for (n_sig, t) in zip(n_sigs, trials):
        t['ntrue'] = np.repeat(n_sig, len(t))
    allt = cy.utils.Arrays.concatenate(trials)
    return allt

def get_color_cycler():
    return cycle(colors)

def plot_ns_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)

    h = hl.hist((allt.ntrue, allt.ns), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(ax.set_ylim(lim))
    ax.plot(lim, lim, **expect_kw)
    ax.set_aspect('equal')

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$n_s$')
    ax.grid()

def plot_gamma_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)
    expect_gamma = tr.sig_injs[0].flux[0].gamma

    h = hl.hist((allt.ntrue, allt.gamma), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(lim)
    ax.set_ylim(1, 4)
    ax.axhline(expect_gamma, **expect_kw)

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$\gamma$')
    ax.grid()

def plot_bkg_trials(
            bg, fig=None, ax=None, 
            label='{} bg trials', 
            label_fit=r'$\chi^2[{:.2f}\mathrm{{dof}},\ \eta={:.3f}]$', 
            color=colors[0],
            density=False,
            bins=50,
        ):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if density:
        h = bg.get_hist(bins=bins).normalize()
    else:
        h = bg.get_hist(bins=bins)
    if label is not None:
        label = label.format(bg.n_total)
    hl.plot1d(ax, h, crosses=True, color=color, label=label)

    # compare with the chi2 fit:
    if hasattr(bg, 'pdf'):
        x = h.centers[0]
        norm = h.integrate().values
        if label_fit is not None:
            label_fit = label_fit.format(bg.ndof, bg.eta)
        if density:
            ax.semilogy(x, bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)
        else:
            ax.semilogy(x, norm * bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)

    ax.set_xlabel(r'TS')
    if density:
        ax.set_ylabel(r'Density')
    else:
        ax.set_ylabel(r'number of trials')
    ax.legend()
        
    return fig, ax

def plot_skymap(
            skymap, outfile=None, figsize=(9, 6),
            vmin=None, vmax=None, label=None, norm=None,
            cmap=cy.plotting.skymap_cmap, plot_gp=True,
            **kwargs
        ):
    """Plot a skymap

    Parameters
    ----------
    skymap : array_like
        The skymap to plot.
    outfile : str, optional
        The output file path to which to plot if provided.
    vmin : float, optional
        The minimum value for the colorbar.
    vmax : float, optional
        The maximum value for the colorbar.
    figsize : tuple, optional
        The figure size to use.
    label : str, optional
        The label for the colorbar.

    Returns
    -------
    fig, ax
        The matplotlib figure and axis.
    """
    fig, ax = plt.subplots(
        subplot_kw=dict(projection='aitoff'), figsize=figsize)
    sp = cy.plotting.SkyPlotter(
        pc_kw=dict(cmap=cmap, vmin=vmin, vmax=vmax, norm=norm),
        **kwargs
    )
    mesh, cb = sp.plot_map(ax, skymap, n_ticks=2)
    kw = dict(color='.5', alpha=.5)
    if plot_gp:
        sp.plot_gp(ax, lw=.5, **kw)
        sp.plot_gc(ax, **kw)
    ax.grid(**kw)
    cb.set_label(label)
    fig.tight_layout()
    if outfile is not None:
        fig.savefig(outfile)

    return fig, ax, sp

## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.0/'

In [ ]:
def get_tr(sindec, gamma, extension=0., cutoff=np.inf, cpus=20, sigsub=True):
    src = cy.utils.sources(0, np.arcsin(sindec), extension=extension, deg=False)
    cutoff_GeV = cutoff * 1e3
    conf = cg.get_ps_conf(
        src=src, gamma=gamma, cutoff_GeV=cutoff_GeV, sigsub=sigsub)

    tr = cy.get_trial_runner(ana=ana, conf=conf, mp_cpus=cpus)
    return tr, src

def load_bkgs(fit=False, max_n=100000, min_dec=-np.inf, max_dec=np.inf):
    base_dir = cg.base_dir + '/ps/trials/DNNC'
    bgfile = '{}/bg.dict'.format(base_dir)
    print('Loading bkg trials')
    bg_trials = np.load(bgfile, allow_pickle=True)['dec']
    print('Creating distribution')
    if fit:
        bgs = {}
        for key, trials in tqdm(bg_trials.items(), total=len(bg_trials)):
            if key > min_dec and key < max_dec:
                bgs[key] = cy.dists.Chi2TSD(trials[:max_n])
    else:
        bgs = {key: cy.dists.TSD(trials) for key, trials in bg_trials.items() if key > min_dec and key < max_dec}
    return bgs

def get_skyscan_tr(dec_deg, bgs, gamma=2.0, cpus=20, nside=128, fit=False, **kwargs):
    """Get Skyscan Trial Runner
    
    Parameters:
    -----------
    gamma : float
        The gamma that is used for signal injection.
    """

    def ts_to_p(dec, ts):
        return cy.dists.ts_to_p(bgs, np.degrees(dec), ts, fit=fit)

    conf = cg.get_ps_conf(src=None, gamma=gamma)
    conf.pop('src')
    conf.update({
        'ana': ana,
        'mp_cpus': cpus,
        'extra_keep': ['energy'],
    })

    inj_src = cy.utils.sources(ra=180, dec=dec_deg, deg=True)
    inj_conf = {
        'src': inj_src,
        'flux': cy.hyp.PowerLawFlux(gamma),
    }

    sstr = cy.get_sky_scan_trial_runner(conf=conf, inj_conf=inj_conf,
                                        min_dec=np.radians(-80),
                                        max_dec=np.radians(80),
                                        mp_scan_cpus=cpus,
                                        nside=nside, ts_to_p=ts_to_p, 
                                        **kwargs)
    return sstr


#### Get Skyscan trialrunners

In [ ]:
fit = True
if fit:
    #bgs = load_bkgs(fit=True, max_n=1000000, min_dec=8.9, max_dec=23.1)
    bgs = load_bkgs(fit=True, max_n=1000000, min_dec=-48, max_dec=23.1)
else:
    bgs = load_bkgs(fit=False)

In [ ]:
%timeit

nside = 64
radius_deg = 7

kwargs_dict = {
    #'Crab': {
    #    'gamma': 3.5, # note this is more like 3.3  3.4
    #    'ns': 160, # more like 180
    #    'dec_deg': 22,
    #},
    'Crab2': {
        'gamma': 3.3, # note this is more like 3.3  3.4
        'ns': 180, # more like 180
        'dec_deg': 22,
    },
    'Crab2_high_ns': {
        'gamma': 3.3, # note this is more like 3.3  3.4
        'ns': 360, # more like 180
        'dec_deg': 22,
    },
    #'3C_454.3': {
    #    'gamma': 3., # note this is more like 3.5 - 3.6
    #    'ns': 200, # more like 213
    #    'dec_deg': 17,
    #},
    '3C_454.3__2': {
        'gamma': 3.5, # note this is more like 3.5 - 3.6
        'ns': 213, # more like 213
        'dec_deg': 17,
    },
    'northern_hotspot': {
        'gamma': 1.86,
        'ns': 28,
        'dec_deg': 10.5,
    },
    'southern_hotspot': {
        'gamma': 2.93,
        'ns': 90.2,
        'dec_deg': -51,
    },
    #'cross_check': {
    #    'gamma': 1.85, 
    #    'ns': 30, 
    #    'dec_deg': 10.5,
    #},
}

tr_dict = {}

for key, kwargs in kwargs_dict.items():
    
    dec_deg = kwargs['dec_deg']
    theta = np.pi/2. - np.deg2rad(dec_deg)
    phi = np.pi
    ipix = hp.query_disc(
        nside=nside, 
        vec=hp.dir2vec(theta=theta, phi=phi), 
        radius=np.deg2rad(radius_deg),
    )
    kwargs['ipix'] = ipix
    theta, phi = hp.pix2ang(nside=nside, ipix=ipix)
    scan_ra = phi
    scan_dec = np.pi/2. - theta
    pixmask = np.zeros(hp.nside2npix(nside), dtype=bool)
    pixmask[ipix] = True
    print('Num pixels:', np.sum(pixmask), len(scan_dec))
    
    tr_dict[key] = get_skyscan_tr(
        dec_deg=kwargs['dec_deg'], 
        gamma=kwargs['gamma'], 
        pixmask=pixmask,
        bgs=bgs,
        nside=nside,
        fit=fit,
        cpus=25,
    )


#### Run signal trials

In [ ]:
from tqdm.notebook import tqdm_notebook as tqdm

n_trials = 1000
seed = 1337
recalculate = False

trials_dict = {}
for key, tr in tr_dict.items():
    print('Running trials for {}'.format(key))

    trials_dict[key] = []

    out_dir = os.path.join(plot_dir, 'scans', key)
    if not os.path.exists(out_dir):
        print('Creating directory: {}'.format(out_dir))
        os.makedirs(out_dir)

    for i in tqdm(range(n_trials), total=n_trials):
        
        seed_i = seed + i
        out_file = os.path.join(out_dir, 'scan_{}_radius_{:3.3f}_{:010d}.pkl'.format(nside, radius_deg, seed_i))

        # load from file
        if os.path.exists(out_file) and not recalculate:
            #print('Loading from {}'.format(out_file))
            with open(out_file, 'rb') as handle:
                trial_compact = pickle.load(handle)

        # recalculate
        else:
            trial = tr.get_one_scan(
                n_sig=kwargs_dict[key]['ns'], poisson=True, logging=True, seed=seed_i,
            )
            trial_compact = {
                'ipix': kwargs_dict[key]['ipix'],
                'mlog10p': trial[0][kwargs_dict[key]['ipix']],
                'ts': trial[1][kwargs_dict[key]['ipix']],
                'ns': trial[2][kwargs_dict[key]['ipix']],
                'gamma': trial[3][kwargs_dict[key]['ipix']],
                'true_dec': np.deg2rad(kwargs_dict[key]['dec_deg']),
                'true_ra': np.pi,
            }
            with open(out_file, 'wb') as f:
                pickle.dump(trial_compact, f, protocol=2)
                
        trials_dict[key].append(trial_compact) 


#### Plot individual trials

In [ ]:
from scipy import stats


def plot_compact_trial(trial_compact):
    skymap = np.zeros(hp.nside2npix(nside))
    skymap[trial_compact['ipix']] = trial_compact['mlog10p']
    print(np.max(trial_compact['mlog10p']), np.min(trial_compact['mlog10p']))
    print(np.isfinite(trial_compact['mlog10p']).all())
    
    fig, ax, sp = plot_skymap(skymap, plot_gp=False, cmap='viridis', label=r'$\log_{10}(p)$')
    theta = np.pi/2. - trial_compact['true_dec']
    phi = trial_compact['true_ra']
    x, y = sp.thetaphi_to_mpl(theta, phi)
    #ax.scatter(x, y, marker='x', color='0.0')
    
    max_ipix = np.argmax(skymap)
    nsigma = stats.norm.isf(10**(-skymap[max_ipix]))
    theta, phi = hp.pix2ang(nside=nside, ipix=max_ipix)
    max_ra = phi
    max_dec = np.pi/2. - theta
    x, y = sp.thetaphi_to_mpl(theta, phi)
    ax.scatter(x, y, marker='x', color='0.0', s=5)
    
    ang_dist = cy.coord.delta_angle(
        zenith1=trial_compact['true_dec'],
        azimuth1=trial_compact['true_ra'],
        zenith2=max_dec,
        azimuth2=max_ra,
        latlon=True,
    )
    ax.set_title('Angular distance: {:3.3f}° | $n\cdot \sigma$: {:3.2f}'.format(np.rad2deg(ang_dist), nsigma))
    
    return fig, ax

fig, ax = plot_compact_trial(trials_dict['cross_check'][8])

fig.savefig('{}/pointing_resolution_trial.png'.format(plot_dir))

#### Compute angular separation and gamma residuals

In [ ]:
ang_dist_dict = {}
sigma_dict = {}
gamma_diff_dict = {}
gamma_dict = {}
mlog10p_dict = {}
ns_dict = {}
for key, tr in tr_dict.items():
    
    ang_dist_list = []
    sigma_list = []
    gamma_diff_list = []
    gamma_list = []
    mlog10p_list = []
    ns_list = []
    for trial_compact in trials_dict[key]:
        
        
        max_idx = np.argmax(trial_compact['mlog10p'])
        max_ipix = trial_compact['ipix'][max_idx]
        theta, phi = hp.pix2ang(nside=nside, ipix=max_ipix)
        max_ra = phi
        max_dec = np.pi/2. - theta
    
        ang_dist = cy.coord.delta_angle(
            zenith1=trial_compact['true_dec'],
            azimuth1=trial_compact['true_ra'],
            zenith2=max_dec,
            azimuth2=max_ra,
            latlon=True,
        )
        ang_dist_list.append(ang_dist)
        sigma_list.append(stats.norm.isf(10**(-np.max(trial_compact['mlog10p']))))
        
        mlog10p_list.append(trial_compact['mlog10p'][max_idx])
        ns_list.append(trial_compact['ns'][max_idx])
        gamma_list.append(trial_compact['gamma'][max_idx])
        gamma_diff_list.append(kwargs_dict[key]['gamma'] - trial_compact['gamma'][max_idx])
        
    ang_dist_dict[key] = np.rad2deg(ang_dist_list)
    sigma_dict[key] = np.array(sigma_list)
    mlog10p_dict[key] = np.array(mlog10p_list)
    ns_dict[key] = np.array(ns_list)
    gamma_dict[key] = np.array(gamma_list)
    gamma_diff_dict[key] = np.array(gamma_diff_list)
    

#### Make Plots

In [ ]:
bins = np.linspace(0, radius_deg, 12)
density = True

fig, ax = plt.subplots(figsize=(9, 6))

for key, ang_dist in ang_dist_dict.items():
    kwargs = kwargs_dict[key]
    ax.hist(
        ang_dist, bins=bins, density=density,
        histtype='step', lw=3,
        label='$\gamma$: {:3.2f} | ns: {:3.1f} | $\delta$: {:3.2f}° | Median: {:3.2f}°'.format(
            kwargs['gamma'], kwargs['ns'], kwargs['dec_deg'], np.median(ang_dist))
            +  ' | {}'.format(key), 
    )

ax.axvline(radius_deg, ls='--', color='0.7', label='Scan radial cutoff')

ax.legend(loc='upper right')
ax.set_xlabel('Opening angle / °')
ax.set_ylabel('Density')
fig.tight_layout()
fig.savefig('{}/pointing_resolution.png'.format(plot_dir))
    
    

In [ ]:
bins = np.linspace(1, 4, 21)
density = True


def get_mask(key, nsigma=2):
    pval = 10**(-mlog10p_dict[key])
    mask = np.logical_and(
        #np.isfinite(mlog10p_dict[key]),
        ns_dict[key] > 0,
        pval < stats.norm.sf(nsigma),
    )
    return mask

for mask_func in [None, get_mask]:
    color_cycler = get_color_cycler()
    fig, ax = plt.subplots(figsize=(9, 6))

    for key in gamma_diff_dict.keys():
        if mask_func is not None:
            mask = get_mask(key)
        else:
            mask = np.ones_like(gamma_dict[key], dtype=bool)
        print('mask', np.sum(mask), len(mask))
        gamma_reco = gamma_dict[key][mask]
        gamma_diff = gamma_diff_dict[key][mask]
        print('gamma', len(gamma_diff))
        
        color = next(color_cycler)
        kwargs = kwargs_dict[key]
        ax.hist(
            gamma_reco, bins=bins, density=density, color=color,
            histtype='step', lw=3,
            label='$\gamma$: {:3.2f} | ns: {:3.1f} | $\delta$: {:3.2f}° | $\Delta\gamma$: {:3.2f} $\pm$ {:3.2f}'.format(
                kwargs['gamma'], kwargs['ns'], kwargs['dec_deg'], np.mean(gamma_diff), np.std(gamma_diff))
                +  ' | {}'.format(key), 
        )
        ax.axvline(kwargs['gamma'], ls='--', color=color)
        ax.axvline(np.median(gamma_reco), ls='-', color=color)

    ax.axvline(np.inf, ls='--', color='0.7', label='True $\gamma$')
    ax.axvline(np.inf, ls='-', color='0.7', label='Median $\gamma_\mathrm{reco}$')

    ax.legend(loc='upper left')
    #ax.set_xlabel('$\Delta \gamma = \gamma_\mathrm{true} - \gamma_\mathrm{reco}$')
    ax.set_xlabel('Spectral index $\gamma$')
    ax.set_ylabel('Density')
    fig.tight_layout()
    if mask_func is None:
        fig.savefig('{}/gamma_resolution.png'.format(plot_dir))
    else:
        fig.savefig('{}/gamma_resolution_masked.png'.format(plot_dir))
    
    